In [ ]:
# drop missing prices and generate returns

prices = get_pricing(stocks, '2014-05-01', '2015-05-01',fields='price').dropna(axis=1)
returns = prices.pct_change().dropna()

# create an new matrix of normalized returns
Y = np.copy(returns.values)
(row, col) = np.shape(Y)
    
#zscored values 
for j in range(0,col):
    mean = np.mean(Y[:,j])
    std = np.std(Y[:,j])
    for i in range(0, row):
        Y[i,j] = (Y[i,j] - mean) / std
        
# compute correlation matrix
corr = np.dot(Y.T, Y) / (row - 1)

# eigen decompose and sort by desc order
eigval,eigv = np.linalg.eig(corr)
sortedindex = eigval.argsort()[::-1]
eigv = eigv[:,sortedindex] 

In [ ]:
cumeigval = np.cumsum(eigval)
plt.plot(cumeigval)

In [ ]:
# take first 60 eigen vectors
eigv = eigv[:, :60]

# convert complex to real numbers and normalize eigenvectors to unit vectors
eigvec = np.zeros(np.shape(eigv))
    
for i in range(0, np.shape(eigv)[0]):
    for j in range(0, np.shape(eigv)[1]):
        eigvec[i,j] = eigv[i,j].real / np.std(returns.values[:,i])

for i in range(0, np.shape(eigvec)[1]):
    eigvec[:,i] = eigvec[:,i] / np.sum(np.abs(eigvec[:,i]))
    
# compute eigen returns            
eigret = np.dot(returns.values, eigvec)

In [ ]:
def fit_OU(ts):
    X = smapi.add_constant(ts[:-1])
    model = smapi.OLS(ts[1:], X)
    results = model.fit()
    b = results.params[1]
    a = results.params[0]
    e = np.std(results.resid)
    k = -math.log(b) * 252
    var = e**2 / (2*k) * (1 - math.exp(-2 * k))
    num = -a * np.sqrt(1-b**2)
    den = (1-b) * np.sqrt(var)
    return num / den, 0. 

In [ ]:
for s in returns:
    # regress stock returns on eigen portfolio returns
    results = smapi.OLS(returns[s].values, smapi.add_constant(eigret)).fit()
    b = results.params
    cumresid = np.cumsum(results.resid)
    score, hf = fit_OU(cumresid) # fit OU 
        
    if abs(score) > 2 and sm.tsa.stattools.adfuller(cumresid)[1] < 0.01:
        plt.plot(np.cumsum(results.resid))
        break